<a href="https://colab.research.google.com/github/Roudranil/competitions/blob/main/dphi/datasprint%2324/Data_Sprint_24_Tinder_Millennial_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>